In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pickle
from tqdm import tqdm

import numpy as np
from scipy import interpolate

sys.path.append("..")
from dm21cm.utils import load_h5_dict
from dm21cm.physics import dt_step

sys.path.append(os.environ['DH_DIR'])
import darkhistory.spec.spectools as spectools
import darkhistory.physics as phys

In [2]:
run_name = 'zf01'
data_dir = f"{os.environ['DM21CM_DATA_DIR']}/tf/{run_name}"

In [3]:
abscs = load_h5_dict(f"../data/abscissas/abscs_{run_name}.h5")

## 1. Get nBs = 1

In [4]:
def get_n1_tf(tf):
    nBs_s = abscs['nBs']
    i_n1 = np.searchsorted(nBs_s, 1.) - 1
    f_left = np.abs(nBs_s[i_n1+1] - 1) / np.abs(nBs_s[i_n1+1] - nBs_s[i_n1])
    f_right = np.abs(nBs_s[i_n1] - 1) / np.abs(nBs_s[i_n1+1] - nBs_s[i_n1])
    return tf[:,:,i_n1,...] * f_left + tf[:,:,i_n1+1,...] * f_right

def get_n1_tf(tf): # zfxxx
    return tf[:,:,6,...]

# def get_n1_tf(tf): # debug_231002
#     return tf[:,:,0,...]
print(get_n1_tf(np.array([[abscs['nBs']]])))

[[1.]]


In [5]:
hep_tf = get_n1_tf(np.load(f'{data_dir}/phot/hep_tf_rxneo.npy')) # rxeo
lep_tf = get_n1_tf(np.load(f'{data_dir}/phot/lep_tf_rxneo.npy'))
lee_tf = get_n1_tf(np.load(f'{data_dir}/phot/lee_tf_rxneo.npy'))
hed_tf = get_n1_tf(np.load(f'{data_dir}/phot/hed_tf_rxneo.npy'))
cmbloss = get_n1_tf(np.load(f'{data_dir}/phot/cmbloss_rxneo.npy'))
lowerbound = get_n1_tf(np.load(f'{data_dir}/phot/lowerbound_rxneo.npy'))

## 2. lowengphot reconstruction

In [6]:
dlnz = abscs['dlnz']
print(f'dlnz = {dlnz}')

dlnz = 0.009950330853168092


In [7]:
for i_rs in range(len(abscs['rs'])):
    for i_x in range(len(abscs['x'])):
        for i in range(len(abscs['photE'])):
            if lep_tf[i_rs,i_x,i,i] > 1e-40:
                break
            lep_tf[i_rs,i_x,i,i] = 1.

## 3. highengphot redshifting and adding cmbloss

In [8]:
dlnphoteng = np.log(5565952217145.328/1e-4)/500
rate = dlnz/dlnphoteng

In [9]:
for i_rs in tqdm(range(len(abscs['rs']))):
    for i_x in range(len(abscs['x'])):
        tf = hep_tf[i_rs, i_x]
        cl = cmbloss[i_rs, i_x]
        rs = abscs['rs'][i_rs]
        lb = lowerbound[i_rs, i_x]
        i_lb = np.searchsorted(abscs['photE'], lb)
        for i_in in range(i_lb, 500):
            tf[i_in][i_in-1] += rate
            tf[i_in][i_in]   -= rate
        cmb_un = spectools.discretize(abscs['photE'], phys.CMB_spec, phys.TCMB(rs)) # unnormalized
        cmb_un_E = cmb_un.toteng()
        #dt = (dlnz/phys.hubble(rs)) # no additional rs step
        for i_in in range(500):
            cmb_E = cl[i_in] # [eV]
            tf[i_in] += (-cmb_E/cmb_un_E) * cmb_un.N

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:04<00:00,  2.15it/s]


## 4. modify DH tf

In [10]:
tfgv_dict = {
    'highengphot' : hep_tf,
    'lowengphot' : lep_tf,
    'lowengelec': lee_tf,
}

In [11]:
for name in ['highengphot', 'lowengphot', 'lowengelec']:
    print(name)
    tf = pickle.load(open(os.environ['DH_DATA_DIR']+f'/{name}_tf_interp.raw', 'rb'))
    tfgv = tfgv_dict[name]

    tf.rs_nodes = np.array([49., 1600.])
    tf._log_interp = False
    tf.dlnz = [dlnz, 0.001, 0.001]
    tf.rs[0] = abscs['rs']

    xHe_s = np.array([0., 1.])
    xHe_grid, xH_grid = np.meshgrid(xHe_s, abscs['x'])
    tf.x[0] = np.stack([xH_grid, xHe_grid], axis=-1)

    tfgv_expanded = np.repeat([tfgv], 2, axis=0)
    tfgv_xhreo = np.einsum('hrxeo -> xhreo', tfgv_expanded)
    tf.grid_vals[0] = tfgv_xhreo
    tf.interp_func[0] = interpolate.RegularGridInterpolator(
        (abscs['x'], xHe_s, abscs['rs']), tf.grid_vals[0]
    )

    pickle.dump(tf, open(f'{data_dir}/{name}_tf_interp.raw', 'wb'))

highengphot


/tmp/ipykernel_1819457/3168852277.py:3: DeprecationWarning: Please use `interp1d` from the `scipy.interpolate` namespace, the `scipy.interpolate.interpolate` namespace is deprecated.
  tf = pickle.load(open(os.environ['DH_DATA_DIR']+f'/{name}_tf_interp.raw', 'rb'))


lowengphot
lowengelec


In [12]:
tf = pickle.load(open(os.environ['DH_DATA_DIR']+'/highengdep_interp.raw', 'rb'))
tfgv = hed_tf

tf.rs_nodes = np.array([49., 1600.])
tf._log_interp = False
tf.dlnz = [dlnz, 0.001, 0.001]
tf.rs[0] = abscs['rs']

xHe_s = np.array([0., 1.])
xHe_grid, xH_grid = np.meshgrid(xHe_s, abscs['x'])
tf.x[0] = np.stack([xH_grid, xHe_grid], axis=-1)

tfgv_expanded = np.repeat([tfgv], 2, axis=0)
tfgv_xhreo = np.einsum('hrxeo -> xhreo', tfgv_expanded)

for i, rs in enumerate(abscs['rs']):
    # dt = (dlnz/phys.hubble(rs)) # old DH dt
    # dt = dts[i, 1] # IDL dt
    dt = dt_step(rs-1, np.exp(dlnz)) # new DH dt
    tfgv_xhreo[:, :, i, :, :] /= dt
tf.grid_vals[0] = tfgv_xhreo
tf.interp_func[0] = interpolate.RegularGridInterpolator(
    (abscs['x'], xHe_s, abscs['rs']), tf.grid_vals[0]
)

pickle.dump(tf, open(f'{data_dir}/highengdep_interp.raw', 'wb'))

/tmp/ipykernel_1819457/1878647830.py:1: DeprecationWarning: Please use `interp1d` from the `scipy.interpolate` namespace, the `scipy.interpolate.interpolate` namespace is deprecated.
  tf = pickle.load(open(os.environ['DH_DATA_DIR']+'/highengdep_interp.raw', 'rb'))


In [13]:
tf = pickle.load(open(os.environ['DH_DATA_DIR']+'/CMB_engloss_interp.raw', 'rb'))
tfgv = cmbloss

tf.rs_nodes = np.array([49., 1600.])
tf._log_interp = False
tf.dlnz = [dlnz, 0.001, 0.001]
tf.rs[0] = abscs['rs']

xHe_s = np.array([0., 1.])
xHe_grid, xH_grid = np.meshgrid(xHe_s, abscs['x'])
tf.x[0] = np.stack([xH_grid, xHe_grid], axis=-1)

tfgv_expanded = np.repeat([tfgv], 2, axis=0)
tfgv_xhre = np.einsum('hrxe -> xhre', tfgv_expanded)

for i, rs in enumerate(abscs['rs']):
    # dt = (dlnz/phys.hubble(rs)) # old DH dt
    # dt = dts[i, 1] # IDL dt
    dt = dt_step(rs-1, np.exp(dlnz)) # new DH dt
    tfgv_xhre[:, :, i, :] /= dt

tf.grid_vals[0] = tfgv_xhre
tf.interp_func[0] = interpolate.RegularGridInterpolator(
    (abscs['x'], xHe_s, abscs['rs']), tf.grid_vals[0]
)

pickle.dump(tf, open(f'{data_dir}/CMB_engloss_interp.raw', 'wb'))

/tmp/ipykernel_1819457/1356181538.py:1: DeprecationWarning: Please use `interp1d` from the `scipy.interpolate` namespace, the `scipy.interpolate.interpolate` namespace is deprecated.
  tf = pickle.load(open(os.environ['DH_DATA_DIR']+'/CMB_engloss_interp.raw', 'rb'))


In [ ]:
! cp -vr $DH_DATA_DIR $DH_DATA_DIR/../DHdata_v1_1_zf01

In [14]:
! ls -l $DM21CM_DATA_DIR/tf/zf01/*.raw

-rw-r--r--. 1 yitians iaifi_lab    7094022 Oct 11 14:36 /n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/CMB_engloss_interp.raw
-rw-r--r--. 1 yitians iaifi_lab   28322034 Oct 11 14:36 /n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/highengdep_interp.raw
-rw-r--r--. 1 yitians iaifi_lab 3538009921 Oct 11 14:35 /n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/highengphot_tf_interp.raw
-rw-r--r--. 1 yitians iaifi_lab 3538013987 Oct 11 14:36 /n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/lowengelec_tf_interp.raw
-rw-r--r--. 1 yitians iaifi_lab 3538009921 Oct 11 14:35 /n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/lowengphot_tf_interp.raw


In [15]:
! cp -v $DM21CM_DATA_DIR/tf/zf01/*.raw /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_zf01/

'/n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/CMB_engloss_interp.raw' -> '/n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_zf01/CMB_engloss_interp.raw'
'/n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/highengdep_interp.raw' -> '/n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_zf01/highengdep_interp.raw'
'/n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/highengphot_tf_interp.raw' -> '/n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_zf01/highengphot_tf_interp.raw'


'/n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/lowengelec_tf_interp.raw' -> '/n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_zf01/lowengelec_tf_interp.raw'
'/n/holyscratch01/iaifi_lab/yitians/dm21cm/DM21cm/data/tf/zf01/lowengphot_tf_interp.raw' -> '/n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_zf01/lowengphot_tf_interp.raw'


In [16]:
! ls -l $DH_DATA_DIR/../DHdata_v1_1_zf01/*.raw

-rw-r--r--. 1 yitians iaifi_lab    7094022 Oct 11 14:36 /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1/../DHdata_v1_1_zf01/CMB_engloss_interp.raw
-rw-r--r--. 1 yitians iaifi_lab 1000361226 Sep 30 12:43 /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1/../DHdata_v1_1_zf01/engloss_ref_tf.raw
-rw-r--r--. 1 yitians iaifi_lab      26589 Sep 30 12:43 /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1/../DHdata_v1_1_zf01/hci_interp.raw
-rw-r--r--. 1 yitians iaifi_lab      26589 Sep 30 12:43 /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1/../DHdata_v1_1_zf01/hep_lb_interp.raw
-rw-r--r--. 1 yitians iaifi_lab 5098010431 Sep 30 12:44 /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1/../DHdata_v1_1_zf01/hep_p12_tf_interp.raw
-rw-r--r--. 1 yitians iaifi_lab 5098010431 Sep 30 12:44 /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1/../DHdata_v1_1_zf01/hep_s11_tf_interp.raw
-rw-r--r--. 1 yitians iaifi_lab   28322034 Oct 11 14:36 /n/holyscratch0